In [1]:
import pandas as pd
import numpy as np
#Import full data set using Pandas:
data=pd.read_csv('/Users/yangxiang/A List/Uchicago/2020_Spring/NLP/Week_1/Food_Inspections.csv')
data.shape

(204389, 17)

In [2]:
data

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2185053,"NEW HONG KONG BUFFET, INC",NEW HONG KONG BUFFET,2510903.0,Restaurant,Risk 1 (High),6249 N MCCORMICK RD,CHICAGO,IL,60659.0,07/25/2018,Complaint,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.995441,-87.712917,"(41.995440602185084, -87.71291737890452)"
1,2365877,STEVE'S 77 ICE CREAM,STEVE'S 77 ICE CREAM,2451374.0,Mobile Frozen Desserts Vendor,Risk 3 (Low),7451 W 100TH PL,BRIDGEVIEW,IL,60455.0,03/17/2020,License,Pass,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,NaN,NaN,NaN
2,2365636,Morgan Park(H/S),Morgan Park(H/S),46251.0,School,Risk 1 (High),1744 W Pryor (11156 S),CHICAGO,IL,60643.0,03/12/2020,Canvass,Pass,NaN,NaN,NaN,NaN
3,2185115,SIMPLY IT,SIMPLY IT,1765716.0,Restaurant,Risk 1 (High),2269 N LINCOLN AVE,CHICAGO,IL,60614.0,07/26/2018,Suspected Food Poisoning,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.923490,-87.645814,"(41.92348957806372, -87.6458144355942)"
4,2222745,MY BUDDY'S,MY BUDDY'S,21943.0,Restaurant,Risk 1 (High),4416 N CLARK ST,CHICAGO,IL,60640.0,09/21/2018,Canvass,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.962160,-87.666219,"(41.962159720050074, -87.6662185738834)"
5,2367144,Chicago Prep Master Tenant LLC,Chicago Prep Master Tenant,2723671.0,Shared Kitchen,Risk 1 (High),5801 W Dickens AVE,CHICAGO,IL,60639.0,04/14/2020,License,Not Ready,NaN,41.917869,-87.770697,"(41.91786924215764, -87.77069733431132)"
6,580885,CESAR'S ON BROADWAY,CESAR'S ON BROADWAY,1492853.0,Restaurant,Risk 1 (High),2924-2926 N BROADWAY,CHICAGO,IL,60657.0,05/11/2011,Short Form Complaint,Pass w/ Conditions,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.935632,-87.644398,"(41.93563185766838, -87.64439807874828)"
7,2200230,TAHOORA SWEETS & BAKERY,TAHOORA SWEETS & BAKERY,1593122.0,Restaurant,Risk 1 (High),2345 W DEVON AVE,CHICAGO,IL,60659.0,07/27/2018,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.997631,-87.689270,"(41.99763133102524, -87.68927000169886)"
8,1584185,THE FIREPLACE INN,THE FIREPLACE INN,2424148.0,Restaurant,Risk 1 (High),1448-1450 N WELLS ST,CHICAGO,IL,60610.0,11/10/2015,License,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.908978,-87.634789,"(41.908977678376715, -87.63478912164865)"
9,1527833,CARRIBEAN JERK PALACE,CARRIBEAN JERK PALACE,2385577.0,Restaurant,Risk 1 (High),3792 W 79TH ST,CHICAGO,IL,60652.0,03/13/2015,License,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.749742,-87.717946,"(41.749741862819484, -87.7179463916298)"


#### multinomial classifier

In [309]:
# Choose 'Pass','Fail','Pass w/ Conditions' ,'Out of Business' from the Results Column as the responce variable
data = data[(data.Results =='Pass') | (data.Results =='Fail') | (data.Results =='Pass w/ Conditions')| (data.Results =='Out of Business')]

In [310]:
data['Results'].value_counts()

Pass                  108500
Fail                   39475
Pass w/ Conditions     29880
Out of Business        17656
Name: Results, dtype: int64

In [311]:
#Making sure that there are no NaNs in "Violations" column
data=data.dropna(subset=['Violations'])

#### Leverage the results of your homework from Week-1 and Week-2 to extract free-form text comments from inspectors
#### Discard the text from “Health Code” – only keep inspectors’ comments

In [312]:
#Using regular expression, parse "Violations" column to select only Comments, create a new column 'Comments'
data=data.reset_index(drop=True)
import re
data['Comments'] =data['Violations'].apply(lambda x: re.findall('Comments[A-Za-z\s\d\:\,\&\;\-\/\.\(\)\"\#\!\@\#\$\%\^\*\_\+\=\\\{\}\[\]\\\'\<\>\?\~\`]+', x))

In [313]:
#Change from list to string for the comment column
data['Comments'] =data['Comments'].apply(lambda x: ' '.join(x))

In [314]:
#Remove "Comments" string
data['Comments']=data['Comments'].apply(lambda x: [re.sub('Comments:','',x)])

In [315]:
#Remove all digital numbers(Remove Health Code)
data['Comments'] =data['Violations'].apply(lambda x: re.findall('\D+', x))

In [316]:
#Change from list to string for the comment column
data['Comments'] =data['Comments'].apply(lambda x: ' '.join(x))

#### Build classification model, predicting the outcome of inspection – your target variable is “Results”

In [317]:
# convert label to a binary numerical variable
data['Results_flag'] = data.Results.map({'Pass':0, 'Fail':1, 'Pass w/ Conditions':2, 'Out of Business':2})

In [318]:
# how to define X and y (from the citation data) for use with COUNTVECTORIZER
X = data.Comments
y = data.Results_flag
print(X.shape)
print(y.shape)

(149590,)
(149590,)


In [319]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.2)

#### Vectorizing our dataset

I choose CountVectorizer as my text pre-processing technique.
CountVectorizer is able to Convert a collection of text documents to a matrix of token counts

In [338]:
# instantiate the vectorizer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
vect = CountVectorizer(stop_words='english',ngram_range=(1, 3),max_df=0.5,min_df=2)
#vect = HashingVectorizer(n_features=15)
#vect = TfidfVectorizer(stop_words='english',ngram_range=(1, 3),max_df=0.5,min_df=2)

In [339]:
# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [340]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<29918x874424 sparse matrix of type '<class 'numpy.int64'>'
	with 7593343 stored elements in Compressed Sparse Row format>

##### Building and evaluating models

###### Naive Bayes Model

In [341]:
from sklearn.naive_bayes import MultinomialNB
# instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

In [342]:
# train and time the model using X_train_dtm
%time nb.fit(X_train_dtm, y_train)

CPU times: user 444 ms, sys: 40.5 ms, total: 484 ms
Wall time: 464 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [343]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [344]:
# calculate accuracy of class predictions
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred_class))

0.8696771174543753


In [345]:
# calculate precision and recall
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95     16868
           1       0.81      0.79      0.80      7212
           2       0.70      0.78      0.74      5838

    accuracy                           0.87     29918
   macro avg       0.82      0.83      0.83     29918
weighted avg       0.87      0.87      0.87     29918



In [346]:
# calculate the confusion matrix
print(confusion_matrix(y_test, y_pred_class))

[[15787   235   846]
 [  402  5689  1121]
 [  168  1127  4543]]


In [347]:
# show default parameters for CountVectorizer
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=2,
                ngram_range=(1, 3), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

###### Logistic Regression Model

In [350]:
# instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression, SGDClassifier
logreg = LogisticRegression()

In [351]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 3min 10s, sys: 7.62 s, total: 3min 18s
Wall time: 2min 39s


/Users/yangxiang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [352]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [354]:
# calculate accuracy of class predictions
print(accuracy_score(y_test, y_pred_class))

0.9463867905608664


In [355]:
# calculate precision and recall
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     16868
           1       0.91      0.90      0.90      7212
           2       0.89      0.88      0.89      5838

    accuracy                           0.95     29918
   macro avg       0.93      0.92      0.93     29918
weighted avg       0.95      0.95      0.95     29918



In [357]:
# calculate the confusion matrix
print(confusion_matrix(y_test, y_pred_class))

[[16700    93    75]
 [  212  6458   542]
 [  146   536  5156]]


###### Support Vector Machine

In [358]:
# instantiate a SVM model
svm = SGDClassifier(max_iter=100, tol=None)

In [359]:
# train the model using X_train_dtm
%time svm.fit(X_train_dtm, y_train)

CPU times: user 58.5 s, sys: 1.41 s, total: 59.9 s
Wall time: 1min


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=100, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=None,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [360]:
# make class predictions for X_test_dtm
y_pred_class = svm.predict(X_test_dtm)

In [361]:
# calculate accuracy of class predictions
print(accuracy_score(y_test, y_pred_class))

0.9434119927802661


In [362]:
# calculate precision and recall
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     16868
           1       0.92      0.89      0.90      7212
           2       0.90      0.88      0.89      5838

    accuracy                           0.94     29918
   macro avg       0.93      0.92      0.92     29918
weighted avg       0.94      0.94      0.94     29918



In [363]:
# calculate the confusion matrix
print(confusion_matrix(y_test, y_pred_class))

[[16673    98    97]
 [  310  6421   481]
 [  231   476  5131]]


 # Logistic Regression Model shows the highest accuracy rate 0.94638, so this is the best model for the analysis